# Simulation

> This module has all the functions required for the inventory simulation functionality

In [ ]:
#| default_exp sdsim

In [ ]:
#| hide
from nbdev.showdoc import *
from datetime import datetime
from datetime import timedelta

import numpy as np
import pandas as pd

from tsmoothie.smoother import ConvolutionSmoother
from tsmoothie.bootstrap import BootstrappingWrapper



In [ ]:
#| export
def foo(): return 10

In [ ]:
#| export
def clip_date(df_full_range):
    """ 
    This function clips the date based on the below criteria
    Note: If the sales qty is zero in the beginning, the date range is trimmed off
    at the begining, till the first positive sales qty is observed
    """

    # check if sales is 0 at the begining. if yes, trim-off the data to start with first postive sales qty
    start = df_full_range.loc[df_full_range['qty'] > 0].index

    if len(start) > 0:
    # 30 is added because, the bootstrap function required 30 day time window.
    
        if start[0] + 30  < len(df_full_range) :
            date = df_full_range.at[start[0], 'date']
        else:
            date = df_full_range.at[0, 'date']
    else:
        date = df_full_range.at[0, 'date']


    return date

In [ ]:
#| export
def Get_Day_Wise_Order_Qty(df_sales_data, MARKET_CODE_ID, DATE_RANGE):
    """ Get daywise sales data for a given SKU in a given date range """


    sub_df = df_sales_data[df_sales_data['mktcode_id'] == MARKET_CODE_ID][[ 'date', 'qty']]
    sub_df.reset_index(drop = True, inplace = True)

    def all_date_dataframe(sub_df):
        """ creates a dataframe having all dates. since the original dataframe has date of ordering only"""
        latest_date    = df_sales_data.date.max()
        begin_date     = latest_date - timedelta(weeks = DATE_RANGE)

        idx = pd.date_range(begin_date, latest_date)
        df_dates = pd.DataFrame(index = idx)
        df_dates.index.name = 'date'

        df_final = pd.merge(df_dates, sub_df, on = ['date'], how = 'left')
        df_final.fillna(0, inplace= True)
        return df_final

    # full range sales qty
    df_full_range = all_date_dataframe(sub_df)
    clipped_date = clip_date(df_full_range)


    mask1 = df_full_range['date'] >= clipped_date
    filtered_series = df_full_range[mask1].reset_index(drop = True)

    return filtered_series

In [ ]:
sales_data     =  pd.read_csv('4571_1Y_sales.csv', parse_dates = ['date'])
#convert market_code_id to integer value
sales_data['mktcode_id'] = sales_data['mktcode_id'].astype('int')
#sort date column chronologically
sales_data.sort_values(by = 'date', ascending= False, inplace=True)
sales_data.head()

,loc_id,mktcode_id,unique_mktcode_name,mktcode_name,fg_id,fg_name,catalog_id,catalog_code,catalog_name,can_be_sold,category_id,category_name,active,date,qty
9576,4571,1011,8513PEC:Safedecor Safari,8513 PEC,3539,FGB0318,6,CAT0002,Safedecor Safari,1,66,0.8 TEX,1,2022-10-13,13
9595,4571,1076,8005PEC:Safedecor Safari,8005 PEC,3277,FGB0056,6,CAT0002,Safedecor Safari,1,66,0.8 TEX,1,2022-10-13,2
9593,4571,8200,8606VNZ:Safedecor Safari,8606 VNZ,20541,FGB0711,6,CAT0002,Safedecor Safari,1,66,0.8 TEX,1,2022-10-13,3
9592,4571,555,11263VCA:Safedecor 1.0mm Master,11263 VCA,3915,FGA0220,4,CAT0001,Safedecor 1.0mm Master,1,93,1.0 TEX 1,1,2022-10-13,5
9591,4571,4701,10062UG+:Safedecor 1.0mm Master,10062 UG+,5115,FGA0591,4,CAT0001,Safedecor 1.0mm Master,1,108,1.0 UG+/SG+,1,2022-10-13,2


In [ ]:
x=Get_Day_Wise_Order_Qty(sales_data, 6250, 52)

In [ ]:
x

,date,qty
0,2021-10-16,5.0
1,2021-10-17,0.0
2,2021-10-18,4.0
3,2021-10-19,2.0
4,2021-10-20,0.0
...,...,...
358,2022-10-09,0.0
359,2022-10-10,14.0
360,2022-10-11,80.0
361,2022-10-12,0.0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()